In [238]:
import warnings

In [239]:
warnings.filterwarnings('ignore')

In [240]:
import pandas as pd 

In [241]:
data=pd.read_csv(r'C:\Users\deyar\OneDrive\Desktop\InsurencepPricePrediction\notebooks\data\insurance.csv')

In [242]:
data.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


In [243]:
data.shape

(1338, 7)

In [244]:
data.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
expenses    0
dtype: int64

In [245]:
data.duplicated().sum()

1

In [246]:
data=data.drop_duplicates()

In [247]:
data.duplicated().sum()

0

In [248]:
cat=[]
num=[]
for i in data.columns:
    if data[i].dtype=="object":
        cat.append(i)
    else:
        num.append(i)

In [249]:
cat

['sex', 'smoker', 'region']

In [250]:
sex=['female', 'male']

In [251]:
region=['southwest', 'southeast', 'northwest', 'northeast']

In [252]:
smoker=['no','yes']

In [253]:
num

['age', 'bmi', 'children', 'expenses']

In [254]:
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler,MinMaxScaler

In [255]:
from sklearn.compose import ColumnTransformer

In [256]:
from sklearn.pipeline import Pipeline

In [257]:
from sklearn.impute import SimpleImputer

In [258]:
data.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


In [259]:
cat

['sex', 'smoker', 'region']

In [351]:

num

['age', 'bmi', 'children']

In [262]:
num.remove('expenses')
num

['age', 'bmi', 'children']

In [263]:
num_pipeline=Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='median')),
                ('scale',MinMaxScaler())])
cat_pipeline=Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',OrdinalEncoder(categories=[sex,smoker,region])),
                ('scale',MinMaxScaler())])
preprocessor=ColumnTransformer([
            ('num_pipeline',num_pipeline,num),
            ('cat_pipeline',cat_pipeline,cat)])

In [264]:
x=data.drop('expenses',axis=1)

In [265]:
y=data['expenses']

In [266]:
m=preprocessor.fit_transform(x)

In [267]:
f=pd.DataFrame(m,columns=x.columns)

In [268]:
f.head()

,age,sex,bmi,children,smoker,region
0,0.021739,0.320755,0.0,0.0,1.0,0.000000
1,0.000000,0.479784,0.2,1.0,0.0,0.333333
2,0.217391,0.458221,0.6,1.0,0.0,0.333333
3,0.326087,0.180593,0.0,1.0,0.0,0.666667
4,0.304348,0.347709,0.0,1.0,0.0,0.666667


In [269]:
f.describe()

,age,sex,bmi,children,smoker,region
count,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000
mean,0.461351,0.395297,0.219147,0.504862,0.204936,0.494640
std,0.305312,0.164438,0.241114,0.500163,0.403806,0.368403
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.195652,0.277628,0.000000,0.000000,0.000000,0.333333
50%,0.456522,0.388140,0.200000,1.000000,0.000000,0.333333
75%,0.717391,0.504043,0.400000,1.000000,0.000000,0.666667
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [270]:
data1=pd.concat([f,y],axis=1)

In [271]:
data1.head()

,age,sex,bmi,children,smoker,region,expenses
0,0.021739,0.320755,0.0,0.0,1.0,0.000000,16884.92
1,0.000000,0.479784,0.2,1.0,0.0,0.333333,1725.55
2,0.217391,0.458221,0.6,1.0,0.0,0.333333,4449.46
3,0.326087,0.180593,0.0,1.0,0.0,0.666667,21984.47
4,0.304348,0.347709,0.0,1.0,0.0,0.666667,3866.86


In [272]:
from sklearn.model_selection import train_test_split

In [273]:
x_trian,x_test,y_train,y_test=train_test_split(f,y,random_state=11,test_size=.2)

In [274]:
from sklearn.ensemble import AdaBoostRegressor

In [275]:
a=AdaBoostRegressor()

In [276]:
a.fit(x_trian,y_train)

AdaBoostRegressor()

In [277]:
p=a.predict(x_test)

In [278]:
from sklearn.metrics import r2_score,mean_absolute_error

In [279]:
r2_score(y_test,p)

0.8489675277959782

In [280]:
import optuna

In [284]:
def objective(trial):
    learning_rate=trial.suggest_float('learning_rate',.001,1,log=True)
    loss=trial.suggest_categorical('loss',['linear','square','exponential'])
    n_estimators=trial.suggest_int('n_estimators',100,1000)
    model=AdaBoostRegressor(n_estimators=n_estimators,
    learning_rate=learning_rate,
    loss=loss
    )
    model.fit(x_trian,y_train)
    p=model.predict(x_test)
    v=r2_score(y_test,p)
    return v

In [285]:
study=optuna.create_study(direction='maximize')

[I 2024-04-19 05:00:23,686] A new study created in memory with name: no-name-8dab0243-fb9c-4b00-a71d-aa77704cfdd6


In [286]:
study.optimize(objective,n_trials=100)

[I 2024-04-19 05:00:24,713] Trial 0 finished with value: 0.8738118723034412 and parameters: {'learning_rate': 0.9431691095417283, 'loss': 'linear', 'n_estimators': 983}. Best is trial 0 with value: 0.8738118723034412.
[I 2024-04-19 05:00:24,977] Trial 1 finished with value: 0.6554551306875172 and parameters: {'learning_rate': 0.1614575651023029, 'loss': 'exponential', 'n_estimators': 168}. Best is trial 0 with value: 0.8738118723034412.
[I 2024-04-19 05:00:25,315] Trial 2 finished with value: 0.8544776268090437 and parameters: {'learning_rate': 0.026312948293508144, 'loss': 'linear', 'n_estimators': 528}. Best is trial 0 with value: 0.8738118723034412.
[I 2024-04-19 05:00:26,429] Trial 3 finished with value: 0.8333557875984653 and parameters: {'learning_rate': 0.004975364137272745, 'loss': 'square', 'n_estimators': 780}. Best is trial 0 with value: 0.8738118723034412.
[I 2024-04-19 05:00:27,096] Trial 4 finished with value: 0.8553418890035747 and parameters: {'learning_rate': 0.0121367

In [287]:
study.best_params

{'learning_rate': 0.0017318756129784008,
 'loss': 'exponential',
 'n_estimators': 116}

In [289]:
study.best_value

0.9148659538632945

In [348]:
ada=AdaBoostRegressor(learning_rate=.0017318756129784008,
 loss='exponential',
 n_estimators=116)

In [349]:
ada.fit(x_trian,y_train)
p=ada.predict(x_test)
r2_score(y_test,p)

0.9141005114751872

In [350]:
ada.score(f,y)

0.8613449444585841

In [290]:
from xgboost import XGBRFRegressor

In [291]:
r1=XGBRFRegressor()

In [292]:
r1.fit(x_trian,y_train)
p=r1.predict(x_test)
r2_score(y_test,p)

0.9118153231697641

In [298]:
r2.score(f,y)

0.9683610283030483

In [314]:
def objective(trial):
    objective=trial.suggest_categorical('objective',['reg:squarederror'])
    learning_rate=trial.suggest_float('learning_rate',.001,1,log=True)
    max_depth=trial.suggest_int('max_depth',1,4)
    min_child_weight=trial.suggest_int('min_child_weight',1,10)
    subsample=trial.suggest_float('subsample',0.4,1)
    colsample_bytree=trial.suggest_float('colsample_bytree',.4,1)
    n_estimators=trial.suggest_int('n_estimators',100,1000)
    model=XGBRFRegressor(n_estimators=n_estimators,
    learning_rate=learning_rate,
    max_depth=max_depth,
    min_child_weight=min_child_weight,
    subsample=subsample,
    colsample_bytree=colsample_bytree,
    objective=objective
    )
    model.fit(x_trian,y_train)
    p=model.predict(x_test)
    v=r2_score(y_test,p)
    return v

In [315]:
study1=optuna.create_study(direction='maximize')

[I 2024-04-19 06:05:23,060] A new study created in memory with name: no-name-ade804e4-771e-4323-9f4f-02f8329579a3


In [316]:
study1.optimize(objective,n_trials=100)

[I 2024-04-19 06:05:23,495] Trial 0 finished with value: -0.011466794434486394 and parameters: {'objective': 'reg:squarederror', 'learning_rate': 0.01450307609779548, 'max_depth': 1, 'min_child_weight': 2, 'subsample': 0.7699106323214673, 'colsample_bytree': 0.41555028212179096, 'n_estimators': 544}. Best is trial 0 with value: -0.011466794434486394.
[I 2024-04-19 06:05:23,613] Trial 1 finished with value: 0.07338422396668332 and parameters: {'objective': 'reg:squarederror', 'learning_rate': 0.05925083983858826, 'max_depth': 4, 'min_child_weight': 5, 'subsample': 0.48261862541066164, 'colsample_bytree': 0.8891030726716054, 'n_estimators': 501}. Best is trial 1 with value: 0.07338422396668332.
[I 2024-04-19 06:05:23,760] Trial 2 finished with value: 0.04213537508759768 and parameters: {'objective': 'reg:squarederror', 'learning_rate': 0.03894565176495049, 'max_depth': 3, 'min_child_weight': 1, 'subsample': 0.7818929400653278, 'colsample_bytree': 0.9273645189778891, 'n_estimators': 951}.

In [317]:
study1.best_params

{'objective': 'reg:squarederror',
 'learning_rate': 0.9951244279464926,
 'max_depth': 3,
 'min_child_weight': 7,
 'subsample': 0.8972000190967084,
 'colsample_bytree': 0.8379622187768934,
 'n_estimators': 400}

In [318]:
study1.best_value

0.8850228943081697

In [322]:
from sklearn.linear_model import ElasticNet,LinearRegression

In [323]:
l=LinearRegression()

In [320]:
e=ElasticNet()

In [324]:
l.fit(x_trian,y_train)
p=l.predict(x_test)
r2_score(y_test,p)

0.8141164268761931

In [331]:
def objective(trial):
    alpha=trial.suggest_float('alpha',.001,1,log=True)
    l1_ratio=trial.suggest_float('l1_ratio',.1,1,log=True)
    max_iter=trial.suggest_int('max_iter',1000,10000)
    model=ElasticNet(alpha=alpha,
    l1_ratio=l1_ratio,
    max_iter=max_iter
    )
    model.fit(x_trian,y_train)
    p=model.predict(x_test)
    v=r2_score(y_test,p)
    return v

In [332]:
study2=optuna.create_study(direction='maximize')

[I 2024-04-19 06:09:38,093] A new study created in memory with name: no-name-6d1f7371-557c-4396-b542-c7ad9f5e8751


In [333]:
study2.optimize(objective,n_trials=100)

[I 2024-04-19 06:09:38,510] Trial 0 finished with value: 0.8142397471956252 and parameters: {'alpha': 0.004580022027769376, 'l1_ratio': 0.8328972239263743, 'max_iter': 4952}. Best is trial 0 with value: 0.8142397471956252.
[I 2024-04-19 06:09:38,513] Trial 1 finished with value: 0.8141554318340828 and parameters: {'alpha': 0.005240940378351492, 'l1_ratio': 0.9658276250664551, 'max_iter': 8276}. Best is trial 0 with value: 0.8142397471956252.
[I 2024-04-19 06:09:38,519] Trial 2 finished with value: 0.7146277471392433 and parameters: {'alpha': 0.11094315704316657, 'l1_ratio': 0.378396831949797, 'max_iter': 2278}. Best is trial 0 with value: 0.8142397471956252.
[I 2024-04-19 06:09:38,523] Trial 3 finished with value: 0.8136609571744884 and parameters: {'alpha': 0.013004843225450816, 'l1_ratio': 0.6805707993772827, 'max_iter': 8448}. Best is trial 0 with value: 0.8142397471956252.
[I 2024-04-19 06:09:38,527] Trial 4 finished with value: 0.8141938782789391 and parameters: {'alpha': 0.001047

In [334]:
study2.best_value

0.8142633161343658

In [335]:
study2.best_params

{'alpha': 0.0025438784156783844,
 'l1_ratio': 0.48274380767581343,
 'max_iter': 2079}

In [336]:
from sklearn.ensemble import RandomForestRegressor

In [337]:
r=RandomForestRegressor()

In [338]:
r.fit(x_trian,y_train)
p=r.predict(x_test)
r2_score(y_test,p)

0.8881483970517412

In [340]:
def objective(trial):
    criterion=trial.suggest_categorical('criterion',['squared_error','absolute_error','poisson'])
    max_depth=trial.suggest_int('max_depth',1,100)
    ccp_alpha=trial.suggest_float('ccp_alpha',0.1,100,log=True)
    n_estimators=trial.suggest_int('n_estimators',100,1000)
    model=RandomForestRegressor(n_estimators=n_estimators,
    max_depth=max_depth,
    ccp_alpha=ccp_alpha,
    criterion=criterion,
    oob_score=True,
    n_jobs=-1
    )
    model.fit(x_trian,y_train)
    p=model.predict(x_test)
    v=r2_score(y_test,p)
    return v

In [341]:
study3=optuna.create_study(direction='maximize')

[I 2024-04-19 06:16:23,462] A new study created in memory with name: no-name-9b32ece5-cf7a-47ea-bcb4-de3947ca21eb


In [342]:
study3.optimize(objective,n_trials=100)

[I 2024-04-19 06:16:40,120] Trial 0 finished with value: 0.896485394302775 and parameters: {'criterion': 'poisson', 'max_depth': 20, 'ccp_alpha': 5.691503146517383, 'n_estimators': 835}. Best is trial 0 with value: 0.896485394302775.
[I 2024-04-19 06:16:41,077] Trial 1 finished with value: 0.9139737137335244 and parameters: {'criterion': 'absolute_error', 'max_depth': 15, 'ccp_alpha': 13.570414270271586, 'n_estimators': 362}. Best is trial 1 with value: 0.9139737137335244.
[I 2024-04-19 06:16:42,868] Trial 2 finished with value: 0.8850609599779329 and parameters: {'criterion': 'absolute_error', 'max_depth': 13, 'ccp_alpha': 1.8980920499362341, 'n_estimators': 730}. Best is trial 1 with value: 0.9139737137335244.
[I 2024-04-19 06:16:43,095] Trial 3 finished with value: 0.8839189927602569 and parameters: {'criterion': 'squared_error', 'max_depth': 40, 'ccp_alpha': 2.1481650346135375, 'n_estimators': 139}. Best is trial 1 with value: 0.9139737137335244.
[I 2024-04-19 06:16:44,433] Trial 4

In [343]:
study3.best_value

0.9253841955224508

In [344]:
study3.best_params

{'criterion': 'absolute_error',
 'max_depth': 5,
 'ccp_alpha': 0.3022712676783935,
 'n_estimators': 900}

In [345]:
reg=RandomForestRegressor(criterion='absolute_error',
 max_depth=5,
 ccp_alpha=0.3022712676783935,
 n_estimators=900)

In [346]:
reg.fit(x_trian,y_train)
p=reg.predict(x_test)
r2_score(y_test,p)

0.9253620502338166

In [347]:
reg.score(f,y)

0.8683996614320667

In [352]:
mean_absolute_error(y_test,p)

2307.0380197674144